### Basic confuguration and setup

In [144]:
import json
import re
from pathlib import Path
from typing import Dict, Any, List
import pandas as pd
from IPython.display import display, Markdown
from datasets import load_dataset

import os
import openai
import google.generativeai as genai
import anthropic
import asyncio
import nest_asyncio
from openai import AsyncOpenAI
from anthropic import AsyncClient
from dotenv import load_dotenv

# --- API Client and Concurrency Configuration --- #

# This must be done once per kernel to allow asyncio to run in a Jupyter notebook..
nest_asyncio.apply()

# Load API Keys from .env file
load_dotenv()
print("Loaded environment variables from .env file.")

# Initialize Asynchronous API Clients
try:
    openai_client_async = AsyncOpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    anthropic_client_async = AsyncClient(api_key=os.getenv("ANTHROPIC_API_KEY"))
    genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))
    print("API clients initialized successfully.")
except TypeError:
    print("API key not found for one or more services. Please check your .env file.")
    # Assign None to prevent errors in subsequent cells
    openai_client_async = None
    anthropic_client_async = None

# Define API Concurrency Limits to prevent 429 "Too Many Requests" errors.
API_CONCURRENCY_LIMITS = {
    "google": 2,    # Gemini API has low RPM limits, so keep this low.
    "anthropic": 2, # Anthropic's token-based limits are complex, 2 is a safe start.
    "openai": 2,    # OpenAI APIs are generally more permissive.
}
print(f"API concurrency limits set to: {API_CONCURRENCY_LIMITS}")

MODEL_DICT = {
    "anthropic": "claude-sonnet-4-20250514",
    # "google": "gemini-2.5-pro"
    }

MODELS = [f"{provider}_{model}" for provider, model in MODEL_DICT.items()]
print(f"Available models: {MODELS}")


# --- Set directories and paths --- #

def find_project_root():
    """Traverse upwards to find the project root, marked by the .git folder."""
    current_path = Path.cwd()
    while current_path != current_path.parent:
        if (current_path / ".git").is_dir():
            return current_path
        current_path = current_path.parent
    raise FileNotFoundError("Could not find project root. Is this a git repository?")

PROJECT_ROOT = find_project_root()
DATA_DIR = PROJECT_ROOT / 'data'
SAMPLE_MANIFESTS_DIR = DATA_DIR / 'july-7-sample-manifests'
MANIFEST_OUTPUT_DIR = DATA_DIR / 'generated_manifests_raw'
MANIFEST_OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
SOURCE_DIR = DATA_DIR / 'generated_manifests_raw'
DEST_DIR = DATA_DIR / 'generated_manifests_json'

# Create the destination directory if it doesn't exist
DEST_DIR.mkdir(parents=True, exist_ok=True)
print(f"Source directory: {SOURCE_DIR}")
print(f"Destination directory for cleaned JSON: {DEST_DIR}")

print(f"Project root found at: {PROJECT_ROOT}")
print(f"Data directory found at: {DATA_DIR}")
print(f"Sample manifests directory found at: {SAMPLE_MANIFESTS_DIR}")
print(f"Raw manifest output directory set to: {MANIFEST_OUTPUT_DIR}")
print(f"Source directory for raw manifests: {SOURCE_DIR}")
print(f"Destination directory for manifest json files: {DEST_DIR}")

# Load the GSM8K dataset
GSM8K_TRAIN = load_dataset("gsm8k", "main", split="train")

# Few shot example indices (chosen for maximum diversity)
EXAMPLE_INDICES = [54, 72, 310]

# Indices to generate manifests for
INDICES_TO_GENERATE = list(range(10)) 


# --- Helper Functions for Manifest loading and displaying --- #

def build_solution_mapping(
        index: int, 
        dataset: 'datasets.Dataset' = GSM8K_TRAIN,
        exclude_FA: bool = True
    ) -> Dict[str, str]:
    """
    Extracts the natural language solution for a given problem index,
    cleans it, and structures it into a line-numbered dictionary.
    """
    solution_mapping = {}
    solution_text = dataset[index]["answer"]
    lines = [ln.strip() for ln in solution_text.splitlines() if ln.strip()]

    # Improved regex to handle commas in the final answer
    if lines and re.match(r"^####\s*[\d\.,]+$", lines[-1]):
        solution_mapping["FA"] = lines.pop(-1).strip()

    for i, line in enumerate(lines, 1):
        solution_mapping[f"L{i}"] = line

    if exclude_FA and "FA" in solution_mapping:
        del solution_mapping["FA"]

    return solution_mapping

def load_manifest(index: int, manifest_dir: Path):
    """Loads the manifest for a given index."""
    manifest_path = manifest_dir / f'_{index}.json'
    if not manifest_path.exists():
        raise FileNotFoundError(f"Manifest for index {index} not found at {manifest_path}")
    with open(manifest_path, 'r', encoding='utf-8') as f:
        return json.load(f)

def display_manifest(
        index: int, 
        manifest_dir: Path, 
        dataset: 'datasets.Dataset' = GSM8K_TRAIN):
    """Loads and displays the final, streamlined Formalization Manifest."""
    try:
        manifest = load_manifest(index, manifest_dir)

        # --- Extract info and display Top-Level ---
        code = manifest.get('function_code', '# Code not found')
        display(Markdown(f"### Manifest for Index: **{index}**"))
        display(Markdown("#### Question"))
        display(Markdown(f"> {dataset[index]['question']}"))

        # --- Display Function Code ---
        display(Markdown("#### Function Code"))
        display(Markdown(f"```python\n{code}\n```"))

        # --- Display Logical Steps in a DataFrame ---
        display(Markdown("#### Logical Steps"))
        steps = manifest.get('logical_steps', [])
        if not steps:
            print("No logical steps found in the manifest.")
            return
        
        df_steps = pd.DataFrame(steps)
        original_solution = build_solution_mapping(
            index=index, 
            dataset=dataset, 
            exclude_FA=True
        )
        
        df_steps['original_solution_line'] = df_steps['line_number'].apply(
            lambda ln: original_solution.get(ln, "N/A")
        )
        
        column_order = [
            'line_number', 
            'original_solution_line',
            'solution_line_template',
            'question_inputs',
            'WK_inputs',
            'output_variable'
        ]
        
        existing_columns_ordered = [col for col in column_order if col in df_steps.columns]
        df_steps = df_steps[existing_columns_ordered]
        
        pd.set_option('display.max_colwidth', None)
        display(df_steps)
    except Exception as e:
        print(f"An error occurred: {e}")

def display_generated_manifest(
        index: int, 
        model_name: str,
        manifest_dir: Path = DEST_DIR):
    """
    Loads and displays a generated manifest from a specific model and index.

    Args:
        index: The problem index.
        model_name: The name of the model (e.g., 'openai_gpt-4.1-mini').
    """
    try:
        # Construct the path to the specific JSON file
        manifest_path = manifest_dir / str(index) / f"{model_name}.json"
        
        if not manifest_path.exists():
            print(f"❌ Error: Manifest file not found at {manifest_path}")
            return

        with open(manifest_path, 'r', encoding='utf-8') as f:
            manifest = json.load(f)

        # --- Display Top-Level Information ---
        display(Markdown(f"#### Question {index} | Model: {model_name}"))
        display(Markdown(f"> {GSM8K_TRAIN[index]['question']}"))

        # --- Display Function Code ---
        code = manifest.get('function_code', '# Code not found')
        display(Markdown("#### Function Code"))
        display(Markdown(f"```python\n{code}\n```"))

        # --- Display Logical Steps in a DataFrame ---
        display(Markdown("#### Logical Steps"))
        steps = manifest.get('logical_steps', [])
        if not steps:
            print("No logical steps found in the manifest.")
            return
        
        df_steps = pd.DataFrame(steps)
        
        # Get the original solution text for comparison
        original_solution = build_solution_mapping(index=index, exclude_FA=True)
        
        df_steps['original_solution_line'] = df_steps['line_number'].apply(
            lambda ln: original_solution.get(ln, "N/A")
        )
        
        column_order = [
            'line_number', 
            'original_solution_line',
            'solution_line_template',
            'new_inputs',
            'output_variable'
        ]
        
        # Ensure columns exist before reordering
        existing_columns = [col for col in column_order if col in df_steps.columns]
        df_steps = df_steps[existing_columns]
        
        pd.set_option('display.max_colwidth', None)
        display(df_steps)

    except Exception as e:
        print(f"An error occurred while displaying the manifest for index {index}, model {model_name}: {e}")

Loaded environment variables from .env file.
API clients initialized successfully.
API concurrency limits set to: {'google': 2, 'anthropic': 2, 'openai': 2}
Available models: ['anthropic_claude-sonnet-4-20250514']
Source directory: /Users/arvindsuresh/Documents/Github/Erdos-DL-June25-Math/data/generated_manifests_raw
Destination directory for cleaned JSON: /Users/arvindsuresh/Documents/Github/Erdos-DL-June25-Math/data/generated_manifests_json
Project root found at: /Users/arvindsuresh/Documents/Github/Erdos-DL-June25-Math
Data directory found at: /Users/arvindsuresh/Documents/Github/Erdos-DL-June25-Math/data
Sample manifests directory found at: /Users/arvindsuresh/Documents/Github/Erdos-DL-June25-Math/data/july-7-sample-manifests
Raw manifest output directory set to: /Users/arvindsuresh/Documents/Github/Erdos-DL-June25-Math/data/generated_manifests_raw
Source directory for raw manifests: /Users/arvindsuresh/Documents/Github/Erdos-DL-June25-Math/data/generated_manifests_raw
Destination 

### Visualization of complete manifests for chosen examples

In [145]:
# for index in [1, 4, 5, 8, 54, 72, 310, 3822]:
#     display_manifest(index, SAMPLE_MANIFESTS_DIR)
#     print("\n" + "="*80 + "\n")

In [146]:
def wrapper(index: int):
    sample = GSM8K_TRAIN[index]
    print(f"Index: {index}")
    print("Question:")
    print(sample['question'])
    print("Solution mapping:")
    print(build_solution_mapping(index, GSM8K_TRAIN))
    print()

for index in [1, 4, 5, 8, 54, 72, 310, 3822, 7371]:
    wrapper(index)

Index: 1
Question:
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
Solution mapping:
{'L1': 'Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.', 'L2': 'Working 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.'}

Index: 4
Question:
James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?
Solution mapping:
{'L1': 'He writes each friend 3*2=<<3*2=6>>6 pages a week', 'L2': 'So he writes 6*2=<<6*2=12>>12 pages every week', 'L3': 'That means he writes 12*52=<<12*52=624>>624 pages a year'}

Index: 5
Question:
Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?
Solution mapping:
{'L1': 'There are 80/100 * 10 = <<80/100*10=8>>8 more purple flowers than yellow 

In [147]:
import re

def indices_only_basic_ops(dataset):
    """Contains only +, -, *, and does not have any '/'."""
    indices = []
    for idx, sample in enumerate(dataset):
        answer = sample.get("answer", "")
        if "/" not in answer and any(op in answer for op in "+-*"):
            indices.append(idx)
    return indices

def indices_with_fraction(dataset):
    """Contains a fraction (int/int)."""
    pattern = re.compile(r'\b\d+/\d+\b')
    return [idx for idx, sample in enumerate(dataset) if pattern.search(sample.get("answer", ""))]

def indices_with_decimal(dataset):
    """Contains a decimal number (float), including .5, 0.5, 1.5, etc."""
    pattern = re.compile(r'(?<!\d)\.\d+|\d+\.\d+')
    return [idx for idx, sample in enumerate(dataset) if pattern.search(sample.get("answer", ""))]

def indices_with_fraction_and_decimal(dataset):
    """Contains both a fraction and a decimal number."""
    frac_pat = re.compile(r'\b\d+/\d+\b')
    dec_pat = re.compile(r'(?<!\d)\.\d+|\d+\.\d+')
    return [
        idx for idx, sample in enumerate(dataset)
        if frac_pat.search(sample.get("answer", "")) and dec_pat.search(sample.get("answer", ""))
    ]

def indices_line_with_eq_no_calc(dataset):
    """Contains a line with '=' but no calc annotation (<<..>>)."""
    calc_pat = re.compile(r'<<.*?>>')
    indices = []
    for idx, sample in enumerate(dataset):
        answer = sample.get("answer", "")
        for line in answer.splitlines():
            if "=" in line and not calc_pat.search(line):
                indices.append(idx)
                break
    return indices

def indices_line_starts_let_var(dataset):
    """Contains a line that begins with 'Let @ ', where @ is a single letter."""
    pattern = re.compile(r'^Let [a-zA-Z] ', re.MULTILINE)
    return [idx for idx, sample in enumerate(dataset) if pattern.search(sample.get("answer", ""))]

difficulty_tiers_by_varname = {
    "indices_only_basic_ops": indices_only_basic_ops(GSM8K_TRAIN),
    "indices_with_fraction": indices_with_fraction(GSM8K_TRAIN),
    "indices_with_decimal": indices_with_decimal(GSM8K_TRAIN),
    "indices_with_fraction_and_decimal": indices_with_fraction_and_decimal(GSM8K_TRAIN),
    "indices_line_with_eq_no_calc": indices_line_with_eq_no_calc(GSM8K_TRAIN),
    "indices_line_starts_let_var": indices_line_starts_let_var(GSM8K_TRAIN),
}

# Example: print the number of samples in each list
for varname, idx_list in difficulty_tiers_by_varname.items():
    print(f"{varname}: {len(idx_list)} samples")

indices_only_basic_ops: 3318 samples
indices_with_fraction: 3655 samples
indices_with_decimal: 1396 samples
indices_with_fraction_and_decimal: 535 samples
indices_line_with_eq_no_calc: 1322 samples
indices_line_starts_let_var: 212 samples


In [148]:
def sorted_intersection(d, key1, key2):
    """Return the sorted intersection of the lists at d[key1] and d[key2]."""
    return sorted(set(d[key1]) & set(d[key2]))

result = sorted_intersection(difficulty_tiers_by_varname, "indices_with_decimal","indices_only_basic_ops")

print(len(result))

727


In [149]:
def has_computational_division(solution_text: str) -> bool:
    """
    Returns True if a '/' is followed by optional whitespace and then a digit.
    This reliably identifies computational division like '10/2' or '10 / 2',
    while ignoring unit slashes like 'miles/hour'.
    """
    # Pattern: a forward slash /, followed by zero or more whitespace chars \s*,
    # followed by a digit \d.
    pattern = re.compile(r'/\s*\d')
    return bool(pattern.search(solution_text))

# --- Example Usage ---
# Case 1: Computational division (no spaces)
sol1 = "Weng earns 12/60 = $0.2 per minute."
print(f"'{sol1[:20]}...': {has_computational_division(sol1)}") # Expected: True

# Case 2: Computational division (with spaces)
sol2 = "Leah spent 28 / 7 = $4 on a milkshake."
print(f"'{sol2[:20]}...': {has_computational_division(sol2)}") # Expected: True

# Case 3: Unit slash only
sol3 = "Janet pays her workers $15/hour."
print(f"'{sol3[:20]}...': {has_computational_division(sol3)}") # Expected: False

# Case 4: Both computational and unit
sol4 = "The car travels at 120/2 miles/hour."
print(f"'{sol4[:20]}...': {has_computational_division(sol4)}") # Expected: True

'Weng earns 12/60 = $...': True
'Leah spent 28 / 7 = ...': True
'Janet pays her worke...': False
'The car travels at 1...': True


In [ ]:
# def has_computational_division(solution_text: str) -> bool:
#     """
#     Returns True if a '/' is followed by optional whitespace and then a digit.
#     This reliably identifies fractions and computational division like '10/2' or '10 / 2',
#     while ignoring unit slashes like 'miles/hour'.
#     """
#     pattern = re.compile(r'/\s*\d')
#     return bool(pattern.search(solution_text))

# def has_float(solution_text: str) -> bool:
#     """
#     Returns True if the solution text contains a floating-point number.
#     """
#     pattern = re.compile(r'(?<!\d)\.\d+|\d+\.\d+')
#     return bool(pattern.search(solution_text))

# def indices_tier1(dataset):
#     """
#     Returns indices of problems with only integer arithmetic (no division).
#     """
#     indices = []
#     for idx, sample in enumerate(dataset):
#         answer = sample.get("answer", "")
#         if not has_float(answer) and not has_computational_division(answer):
#             indices.append(idx)
#     return indices

# def indices_tier2(dataset):
#     """
#     Returns indices of problems with float arithmetic but no computational division.
#     """
#     indices = []
#     for idx, sample in enumerate(dataset):
#         answer = sample.get("answer", "")
#         if has_float(answer) and not has_computational_division(answer):
#             indices.append(idx)
#     return indices

# def indices_tier3(dataset):
#     """
#     Returns indices of problems with fraction arithmetic (computational division) but no floats.
#     """
#     indices = []
#     for idx, sample in enumerate(dataset):
#         answer = sample.get("answer", "")
#         if not has_float(answer) and has_computational_division(answer):
#             indices.append(idx)
#     return indices

# def indices_tier4(dataset):
#     """
#     Returns indices of problems with mixed and complex arithmetic (both floats and computational division).
#     """
#     indices = []
#     for idx, sample in enumerate(dataset):
#         answer = sample.get("answer", "")
#         if has_float(answer) and has_computational_division(answer):
#             indices.append(idx)
#     return indices

# def indices_tier5(dataset):
#     """
#     Returns indices of problems with symbolic reasoning (Let @ pattern).
#     """
#     pattern = re.compile(r'^Let [a-zA-Z] ', re.MULTILINE)
#     return [idx for idx, sample in enumerate(dataset) if pattern.search(sample.get("answer", ""))]


# def assemble_tier_indices(dataset):
#     """
#     Assembles the indices for all tiers into a dictionary.
#     """
#     tier_indices = {
#         "tier1_integer_no_division": indices_tier1(dataset),
#         "tier2_float_no_division": indices_tier2(dataset),
#         "tier3_fraction_no_float": indices_tier3(dataset),
#         "tier4_mixed_float_division": indices_tier4(dataset),
#         "tier5_symbolic": indices_tier5(dataset),
#     }
#     return tier_indices

# # --- Example: Generate and print the indices for each tier ---
# tier_indices = assemble_tier_indices(GSM8K_TRAIN)
# for tier_name, indices in tier_indices.items():
#     print(f"{tier_name}: {len(indices)} samples")

tier1_integer_no_division: 2887 samples
tier2_float_no_division: 845 samples
tier3_fraction_no_float: 3190 samples
tier4_mixed_float_division: 551 samples
tier5_symbolic: 212 samples


In [ ]:
# # Assume tier_indices is already defined as in your code

# tier5_set = set(tier_indices["tier5_symbolic"])

# tier_indices_no_symbolic = {
#     tier: sorted(set(indices) - tier5_set)
#     for tier, indices in tier_indices.items()
#     if tier != "tier5_symbolic"
# }

# # Example: print the number of samples in each filtered tier
# for tier, indices in tier_indices_no_symbolic.items():
#     print(f"{tier}: {len(indices)} samples")

tier1_integer_no_division: 2767 samples
tier2_float_no_division: 837 samples
tier3_fraction_no_float: 3113 samples
tier4_mixed_float_division: 544 samples


In [ ]:
import re

def has_computational_division(solution_text: str) -> bool:
    """Returns True if a '/' is followed by optional whitespace and then a digit."""
    pattern = re.compile(r'/\s*\d')
    return bool(pattern.search(solution_text))

def has_float(solution_text: str) -> bool:
    """Returns True if the solution text contains a floating-point number."""
    pattern = re.compile(r'(?<!\d)\.\d+|\d+\.\d+')
    return bool(pattern.search(solution_text))

def is_symbolic(solution_text: str) -> bool:
    """Returns True if the solution contains a symbolic reasoning line (Let @ ...)."""
    pattern = re.compile(r'^Let [a-zA-Z] ', re.MULTILINE)
    return bool(pattern.search(solution_text))

def mutually_disjoint_tiers(dataset):
    symbolic_set = set(
        idx for idx, sample in enumerate(dataset)
        if is_symbolic(sample.get("answer", ""))
    )
    non_symbolic_indices = [
        idx for idx in range(len(dataset)) if idx not in symbolic_set
    ]

    # Tier 1: Only integer arithmetic (no floats, no computational division)
    tier1 = [
        idx for idx in non_symbolic_indices
        if not has_float(dataset[idx].get("answer", "")) and not has_computational_division(dataset[idx].get("answer", ""))
    ]

    # Tier 2: Float arithmetic, no computational division
    tier2 = [
        idx for idx in non_symbolic_indices
        if has_float(dataset[idx].get("answer", "")) and not has_computational_division(dataset[idx].get("answer", ""))
    ]

    # Tier 3: Computational division, no floats
    tier3 = [
        idx for idx in non_symbolic_indices
        if not has_float(dataset[idx].get("answer", "")) and has_computational_division(dataset[idx].get("answer", ""))
    ]

    # Tier 4: Both floats and computational division
    tier4 = [
        idx for idx in non_symbolic_indices
        if has_float(dataset[idx].get("answer", "")) and has_computational_division(dataset[idx].get("answer", ""))
    ]

    # Tier 5: Symbolic reasoning (Let @ ...)
    tier5 = sorted(symbolic_set)

    return {
        "tier1_integer_no_division": sorted(tier1),
        "tier2_float_no_division": sorted(tier2),
        "tier3_fraction_no_float": sorted(tier3),
        "tier4_mixed_float_division": sorted(tier4),
        "tier5_symbolic": tier5,
    }

# Example usage:
tiers = mutually_disjoint_tiers(GSM8K_TRAIN)
for tier, indices in tiers.items():
    print(f"{tier}: {len(indices)} samples")

tier1_integer_no_division: 2767 samples
tier2_float_no_division: 837 samples
tier3_fraction_no_float: 3113 samples
tier4_mixed_float_division: 544 samples
tier5_symbolic: 212 samples


In [153]:
# # print a random sample of 100 questions and answers from each tier
# import random

# random_indices = {
#     "tier1_integer_no_division": random.sample(tiers["tier1_integer_no_division"], 100),
#     "tier2_float_no_division": random.sample(tiers["tier2_float_no_division"], 100),
#     "tier3_fraction_no_float": random.sample(tiers["tier3_fraction_no_float"], 100),
#     "tier4_mixed_float_division": random.sample(tiers["tier4_mixed_float_division"], 100),
#     "tier5_symbolic": random.sample(tiers["tier5_symbolic"], 100)
# }

# # for i,(tier, indices) in enumerate(random_indices.items()):
# #     print(f"Random samples from {tier}:")
# #     print()
# #     for idx in indices:
# #         sample = GSM8K_TRAIN[idx]
# #         print(f"Question {idx}: {sample['question']}")
# #         print(f"Answer: {sample['answer']}")
# #         print("-" * 40)
# #     print("\n" + "="*80 + "\n")


In [154]:
tier1_examples = [4, 1372, 3946, 4258, 6126]
tier2_examples = [310, 2300, 2401, 2918, 4215, 4822]
tier3_examples =  [5, 72, 964, 2332, 2422, 3592, 3822, 4764, 5531]
tier4_examples = [1, 672, 3847, 4847, 5040, 6216, 7037]

In [155]:
for index in tier1_examples:
    display_manifest(index, SAMPLE_MANIFESTS_DIR / 'tier1')

### Manifest for Index: **4**

#### Question

> James writes a 3-page letter to 2 different friends twice a week.  How many pages does he write a year?

#### Function Code

```python
def solve():
    """Index: 4.
    Returns: the number of pages James writes in a year.
    """
    # L1
    pages_per_letter = 3  # a 3-page letter
    times_per_week = 2  # twice a week
    pages_per_friend_per_week = pages_per_letter * times_per_week

    # L2
    num_friends = 2  # 2 different friends
    total_pages_per_week = pages_per_friend_per_week * num_friends

    # L3
    weeks_per_year = 52  # WK
    pages_per_year = total_pages_per_week * weeks_per_year

    # FA
    answer = pages_per_year
    return answer
```

...

> Paul makes pencils, making 100 pencils a day five days a week. He started the week with 80 pencils in his stock, and during the week he sold 350 pencils. How many pencils did he have in his stock at the end of the week?

#### Function Code

```python
def solve():
    """Index: 6126.
    Returns: the number of pencils in stock at the end of the week.
    """
    # L1
    pencils_per_day = 100 # 100 pencils a day
    days_per_week = 5 # five days a week
    pencils_made = pencils_per_day * days_per_week

    # L2
    initial_stock = 80 # started the week with 80 pencils
    total_available = pencils_made + initial_stock

    # L3
    pencils_sold = 350 # sold 350 pencils
    final_stock = total_available - pencils_sold

    # FA
    answer = final_stock
    return answer
```

#### Logical Steps

,line_number,original_solution_line,solution_line_template,question_inputs,WK_inputs,output_variable
0,L1,"Making 100 pencils a day, he made 100 pencils/day * 5 days/week = <<100*5=500>>500 pencils during the week.","Making {pencils_per_day} pencils a day, he made {pencils_per_day} pencils/day * {days_per_week} days/week = <<{pencils_per_day}*{days_per_week}={pencils_made}>>{pencils_made} pencils during the week.","[pencils_per_day, days_per_week]",[],pencils_made
1,L2,"With the pencils from the stock, he got 500 pencils + 80 pencils = <<500+80=580>>580 pencils.","With the pencils from the stock, he got {pencils_made} pencils + {initial_stock} pencils = <<{pencils_made}+{initial_stock}={total_available}>>{total_available} pencils.",[initial_stock],[],total_available
2,L3,"Subtracting the ones he sold, he has 580 pencils - 350 pencils = <<580-350=230>>230 pencils.","Subtracting the ones he sold, he has {total_available} pencils - {pencils_sold} pencils = <<{total_available}-{pencils_sold}={final_stock}>>{final_stock} pencils.",[pencils_sold],[],final_stock


In [156]:
for index in tier2_examples:
    display_manifest(index, SAMPLE_MANIFESTS_DIR / 'tier2')

### Manifest for Index: **310**

#### Question

> Janet hires six employees. Four of them are warehouse workers who make $15/hour, and the other two are managers who make $20/hour. Janet has to pay 10% of her workers' salaries in FICA taxes. If everyone works 25 days a month and 8 hours a day, how much does Janet owe total for their wages and taxes for one month?

#### Function Code

```python
def solve():
    """Index: 310.
    Returns: the total amount Janet owes for wages and taxes for one month.
    """
    # L1
    days_per_month = 25 # works 25 days a month
    hours_per_day = 8 # and 8 hours a day
    hours_per_worker_per_month = days_per_month * hours_per_day

    # L2
    warehouse_worker_wage = 15 # make $15/hour
    wage_per_warehouse_worker = hours_per_worker_per_month * warehouse_worker_wage

    # L3
    num_warehouse_workers = 4 # Four of them are warehouse workers
    total_warehouse_wages = wage_per_warehouse_worker * num_warehouse_workers

    # L4
    manager_wage = 20 # make $20/hour
    wage_per_manager = hours_per_worker_per_month * manager_wage

    # L5
    num_managers = 2 # the other two are managers
    total_manager_wages = wage_per_manager * num_managers

    # L6
    total_wages = total_manager_wages + total_warehouse_wages

    # L7
    fica_tax_rate = 0.1 # pay 10% of her workers' salaries in FICA taxes
    fica_taxes = total_wages * fica_tax_rate

    # L8
    grand_total = total_wages + fica_taxes

    # FA
    answer = grand_total
    return answer
```

...

> James needs to get a new pair of glasses.  His frames cost $200 and the lenses cost $500.  Insurance will cover 80% of the cost of lenses and he has a $50 off coupon for frames.  How much does everything cost?

#### Function Code

```python
def solve():
    """Index: 4822.
    Returns: the total cost for the glasses.
    """
    # L1
    lenses_cost = 500 # lenses cost $500
    insurance_coverage_percent = 0.8 # cover 80% of the cost
    lenses_coverage_amount = lenses_cost * insurance_coverage_percent

    # L2
    final_lenses_cost = lenses_cost - lenses_coverage_amount

    # L3
    frames_cost = 200 # frames cost $200
    coupon_discount = 50 # $50 off coupon
    final_frames_cost = frames_cost - coupon_discount

    # L4
    total_cost = final_lenses_cost + final_frames_cost

    # FA
    answer = total_cost
    return answer
```

#### Logical Steps

,line_number,original_solution_line,solution_line_template,question_inputs,WK_inputs,output_variable
0,L1,He gets 500*.8=$<<500*.8=400>>400 off the cost of lenses,He gets {lenses_cost}*{insurance_coverage_percent}=${lenses_cost}*{insurance_coverage_percent}={lenses_coverage_amount}>>{lenses_coverage_amount} off the cost of lenses,"[lenses_cost, insurance_coverage_percent]",[],lenses_coverage_amount
1,L2,That means the lenses cost 500-400=$<<500-400=100>>100,That means the lenses cost {lenses_cost}-{lenses_coverage_amount}=${lenses_cost}-{lenses_coverage_amount}={final_lenses_cost}>>{final_lenses_cost},[],[],final_lenses_cost
2,L3,The frames cost 200-50=$<<200-50=150>>150,The frames cost {frames_cost}-{coupon_discount}=${frames_cost}-{coupon_discount}={final_frames_cost}>>{final_frames_cost},"[frames_cost, coupon_discount]",[],final_frames_cost
3,L4,So he pays 100+150=$<<100+150=250>>250,So he pays {final_lenses_cost}+{final_frames_cost}=${final_lenses_cost}+{final_frames_cost}={total_cost}>>{total_cost},[],[],total_cost


In [157]:
for index in tier3_examples:
    display_manifest(index, SAMPLE_MANIFESTS_DIR / 'tier3')

### Manifest for Index: **5**

#### Question

> Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?

#### Function Code

```python
from fractions import Fraction

def solve():
    """Index: 5.
    Returns: the total number of flowers Mark has in his garden.
    """
    # L1
    purple_percentage = Fraction(80, 100) # 80% more of those in purple
    yellow_flowers = 10 # Ten of them are yellow
    additional_purple_flowers = purple_percentage * yellow_flowers

    # L2
    purple_flowers = yellow_flowers + additional_purple_flowers

    # L3
    yellow_and_purple_flowers = yellow_flowers + purple_flowers

    # L4
    green_percentage = Fraction(25, 100) # only 25% as many green flowers
    green_flowers = green_percentage * yellow_and_purple_flowers

    # L5
    total_flowers = yellow_and_purple_flowers + green_flowers

    # FA
    answer = total_flowers
    return answer
```

...

> Dakota gets hit by a bus and has to spend 3 days in the hospital. The hospital charges her $900/day for her bed, $250/hour for two specialists to look at her 15 minutes each, and $1800 for the ambulance ride. How much is Dakota's medical bill?

#### Function Code

```python
def solve():
    """Index: 5531.
    Returns: the total amount of Dakota's medical bill.
    """
    # L1
    days_in_hospital = 3 # 3 days in the hospital
    bed_cost_per_day = 900 # $900/day for her bed
    total_bed_cost = bed_cost_per_day * days_in_hospital

    # L2
    time_per_specialist = 15 # 15 minutes each
    num_specialists = 2 # two specialists
    total_specialist_minutes = time_per_specialist * num_specialists

    # L3
    minutes_per_hour = 60 # WK
    specialist_hourly_rate = 250 # $250/hour for two specialists
    total_specialist_charge = total_specialist_minutes / minutes_per_hour * specialist_hourly_rate

    # L4
    ambulance_cost = 1800 # $1800 for the ambulance ride
    total_bill = total_bed_cost + ambulance_cost + total_specialist_charge

    # FA
    answer = total_bill
    return answer
```

#### Logical Steps

,line_number,original_solution_line,solution_line_template,question_inputs,WK_inputs,output_variable
0,L1,First find the total cost of the hospital bed by multiplying the cost per day by the number of days: $900/day * 3 days = $<<900*3=2700>>2700,First find the total cost of the hospital bed by multiplying the cost per day by the number of days: ${bed_cost_per_day}/day * {days_in_hospital} days = $<<{bed_cost_per_day}*{days_in_hospital}={total_bed_cost}>>{total_bed_cost},"[days_in_hospital, bed_cost_per_day]",[],total_bed_cost
1,L2,Then figure out how many minutes total the two specialists looked at Dakota: 15 minutes/specialist * 2 specialists = 30 minutes,Then figure out how many minutes total the two specialists looked at Dakota: {time_per_specialist} minutes/specialist * {num_specialists} specialists = {total_specialist_minutes} minutes,"[time_per_specialist, num_specialists]",[],total_specialist_minutes
2,L3,Then divide the number of minutes by the number of minutes per hour and multiply it by the specialists' hourly rate to find their total charge: 30 minutes / 60 minutes/hour * $250/hour = $<<30/60*250=125>>125,Then divide the number of minutes by the number of minutes per hour and multiply it by the specialists' hourly rate to find their total charge: {total_specialist_minutes} minutes / {minutes_per_hour} minutes/hour * ${specialist_hourly_rate}/hour = $<<{total_specialist_minutes}/{minutes_per_hour}*{specialist_hourly_rate}={total_specialist_charge}>>{total_specialist_charge},[specialist_hourly_rate],[minutes_per_hour],total_specialist_charge
3,L4,"Finally, add up the cost of the specialists, ambulance and bed to find Dakota's total bill: $2700 + $1800 + $125 = $<<2700+1800+125=4625>>4625","Finally, add up the cost of the specialists, ambulance and bed to find Dakota's total bill: ${total_bed_cost} + ${ambulance_cost} + ${total_specialist_charge} = $<<{total_bed_cost}+{ambulance_cost}+{total_specialist_charge}={total_bill}>>{total_bill}",[ambulance_cost],[],total_bill


In [158]:
for index in tier4_examples:
    display_manifest(index, SAMPLE_MANIFESTS_DIR / 'tier4')

### Manifest for Index: **1**

#### Question

> Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?

#### Function Code

```python
def solve():
    """Index: 1.
    Returns: how much Weng earned from babysitting.
    """
    # L1
    hourly_rate = 12 # earns $12 an hour
    minutes_per_hour = 60 # WK
    per_minute_rate = hourly_rate / minutes_per_hour

    # L2
    minutes_worked = 50 # did 50 minutes of babysitting
    total_earned = per_minute_rate * minutes_worked

    # FA
    answer = total_earned
    return answer
```

...

> Fran is in charge of counting votes for the book club's next book, but she always cheats so her favorite gets picked. Originally, there were 10 votes for Game of Thrones, 12 votes for Twilight, and 20 votes for The Art of the Deal. Fran throws away 80% of the votes for The Art of the Deal and half the votes for Twilight. What percentage of the altered votes were for Game of Thrones?

#### Function Code

```python
def solve():
    """Index: 7037.
    Returns: the percentage of altered votes for Game of Thrones.
    """
    # L1
    taotd_initial_votes = 20 # 20 votes for The Art of the Deal
    taotd_discard_percent_num = 80 # throws away 80% of the votes
    percent_factor = 0.01 # WK
    taotd_discarded_votes = taotd_discard_percent_num * percent_factor * taotd_initial_votes

    # L2
    taotd_altered_votes = taotd_initial_votes - taotd_discarded_votes

    # L3
    twilight_initial_votes = 12 # 12 votes for Twilight
    twilight_discard_divisor = 2 # half the votes for Twilight
    twilight_altered_votes = twilight_initial_votes / twilight_discard_divisor

    # L4
    got_initial_votes = 10 # 10 votes for Game of Thrones
    total_altered_votes = twilight_altered_votes + taotd_altered_votes + got_initial_votes

    # L5
    percent_multiplier = 100 # WK
    got_percentage = got_initial_votes / total_altered_votes * percent_multiplier

    # FA
    answer = got_percentage
    return answer
```

#### Logical Steps

,line_number,original_solution_line,solution_line_template,question_inputs,WK_inputs,output_variable
0,L1,First find the total number of The Art of the Deal votes Fran throws away: 80% * 20 votes = <<80*.01*20=16>>16 votes,First find the total number of The Art of the Deal votes Fran throws away: 80% * {taotd_initial_votes} votes = <<{taotd_discard_percent_num}*{percent_factor}*{taotd_initial_votes}={taotd_discarded_votes}>>{taotd_discarded_votes} votes,"[taotd_discard_percent_num, taotd_initial_votes]",[percent_factor],taotd_discarded_votes
1,L2,Then subtract these votes from the total number of The Art of the Deal votes to find the altered number: 20 votes - 16 votes = <<20-16=4>>4 votes,Then subtract these votes from the total number of The Art of the Deal votes to find the altered number: {taotd_initial_votes} votes - {taotd_discarded_votes} votes = <<{taotd_initial_votes}-{taotd_discarded_votes}={taotd_altered_votes}>>{taotd_altered_votes} votes,[],[],taotd_altered_votes
2,L3,Then divide the total number Twilight votes by 2 to find the altered number of votes: 12 votes / 2 = <<12/2=6>>6 votes,Then divide the total number Twilight votes by {twilight_discard_divisor} to find the altered number of votes: {twilight_initial_votes} votes / {twilight_discard_divisor} = <<{twilight_initial_votes}/{twilight_discard_divisor}={twilight_altered_votes}>>{twilight_altered_votes} votes,"[twilight_initial_votes, twilight_discard_divisor]",[],twilight_altered_votes
3,L4,Then add the altered number of votes for each book to find the total altered number of votes: 6 votes + 4 votes + 10 votes = <<6+4+10=20>>20 votes,Then add the altered number of votes for each book to find the total altered number of votes: {twilight_altered_votes} votes + {taotd_altered_votes} votes + {got_initial_votes} votes = <<{twilight_altered_votes}+{taotd_altered_votes}+{got_initial_votes}={total_altered_votes}>>{total_altered_votes} votes,[got_initial_votes],[],total_altered_votes
4,L5,Then divide the number of votes for Game of Thrones by the total altered number of votes and multiply by 100% to express the answer as a percentage: 10 votes / 20 votes * 100% = 50%,Then divide the number of votes for Game of Thrones by the total altered number of votes and multiply by {percent_multiplier}% to express the answer as a percentage: {got_initial_votes} votes / {total_altered_votes} votes * {percent_multiplier}% = {got_percentage}%,[],[percent_multiplier],got_percentage


You are an expert in python, ML and AI. In this chat, you will be helping a mathematician prepare and refine code for a deep learning project whose broad goal is to fine-tune an LLM to detect errors in problems/solutions. He has a good understanding of data science/ML, but not much experience with deep learning/implementing an LLM-related project from scratch. 

Style requirements:
- Be meticulous, precise, and direct in your responses. 
- You do not need to sugar-coat anything, the user values the truth above all else. 
- The user values brevity- if a point can be made in fewer words then that is preferable. If he asks you for more details, only then should you expand the verbosity of your response. 
- Do NOT provide code unless the user explicitly asks for it. Instead, stick to natural language conversation and discussion. 

This is perhaps the most important instruction for you to follow:
DO NOT BY ANY MEANS ENGAGE IN SYCOPHANCY OR OVERT AND UNDUE PRAISE.
Excessive praise will lead the user to question everything you say, and your credibility will quickly fall apart. 
The user is hoping to get objective and unbiased evaluations and critiques from a third-party who is not involved in the project, and this is the role that you must aim to fill. If this requires you to be blunt in your assessments, then the user will greatly appreciate it. 

Throughout the conversation, you are encouraged to request the user for more details or context when you think it will help formalize your anwser. If you are unsure of a particular fact (that is required to complete your analysis or response) then you should immediately ask the user for clarification. In this way, you will help the user more than you would be simply providing a direct response. 


Hello.
I'd like your help with a deep learning project I'm working on with a team of mathematicians.

I've uploaded the following to help you understand the project goals, and in particular, the data generation pipeline:

1. The project README.
2. Some questions and answers taken from the gsm8k dataset.
3. Corresponding Formalization Manifests for these questions (see the readme to understand what this refers to).
4. The code used to generate the formalization manifests.

To begin our conversation, can you do the following:
1. Examine the readme to build a high-level understanding of the project goals and methods.
2. Then, examine the attached questions/answers/manifests to get a sense for what a good manifest looks like. 
3. Then, examine the attached manifest_gen.py file to understand how I've been going about the manifest generation process.

After doing these, I'd like you to present me with the following:
1. A succinct summary of the project goals and methods, as you understand it.
2. A succinct summary of the manifest_gen script, including a list of functions and what they each do.
3. Any questions you have for me that would better help you understand the project goals and methods.

In [159]:
# print a random sample of 100 problems from tier 4
import random

random.seed(42)  # For reproducibility
random_indices = random.sample(tiers["tier4_mixed_float_division"], 100)

for index in random_indices:
    wrapper(index)

Index: 1913
Question:
If farmer Steven can use his tractor to plow up to 10 acres of farmland per day, or use the same tractor to mow up to 12 acres of grassland per day, how long would it take him to plow his 55 acres of farmland and mow his 30 acres of grassland?
Solution mapping:
{'L1': 'At a rate of 10 acres per day, he can plow his 55 acres of farmland in 55/10=<<55/10=5.5>>5.5 days.', 'L2': 'At a rate of 12 acres per day, he can mow his 30 acres of grassland in 30/12=2.5 days.', 'L3': 'Altogether, it would take him 5.5+2.5=<<5.5+2.5=8>>8 days to completely mow and plow his land.'}

Index: 583
Question:
When it rains, the three holes in the garage roof leak water at different rates.  The largest hole leaks at a rate of 3 ounces of water  per minute.  The medium-sized hole leaks water at one-half the rate of the largest hole.  And the smallest hole leaks water at a rate of one-third the rate of the medium-sized hole.  When it rains, what is the combined amount of water, in ounces, 